In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- Configuration ---
# We'll use a 60-day window
WINDOW_SIZE = 60

# Define our directories
MODELS_DIR = "models"
DATA_DIR = "data"

# Ensure the models directory exists
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)
    print(f"Created directory: {MODELS_DIR}")
else:
    print(f"Directory already exists: {MODELS_DIR}")


# --- 1. Data Preprocessing Function ---
def process_data(file_path, window_size):
    """
    Loads data, scales it, and creates time-series windows (X, y).
    """
    print(f"Loading and processing data from {file_path}...")
    
    try:
        df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None, None, None
        
    # Features to use for training
    features = ['Max_Temp', 'Min_Temp', 'Rainfall', 'Price']
    data = df[features]
    
    # Scale the data to [0, 1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    # Create Windows (Sequences)
    X = []
    y = []
    
    for i in range(window_size, len(scaled_data)):
        # X: The previous 'window_size' days of data
        X.append(scaled_data[i-window_size:i])
        
        # y: The 'Price' on the 'i'-th day (index 3)
        y.append(scaled_data[i, 3]) 
        
    X, y = np.array(X), np.array(y)
    
    print(f"Data shape: X={X.shape}, y={y.shape}")
    
    return X, y, scaler

# --- 2. Model Building Function ---
def build_lstm_model(input_shape):
    """
    Defines and compiles the LSTM model architecture.
    """
    model = Sequential()
    
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(units=25))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

print("Helper functions and configuration are ready.")

Directory already exists: models
Helper functions and configuration are ready.


In [2]:
# --- 1. SETTINGS FOR MADIKERI ---
model_name = "madikeri"
file_path = os.path.join(DATA_DIR, "madikeri_merged.csv")

print(f"\n--- Training Model for: {model_name.upper()} ---")

# Step 1: Process the data
X_madikeri, y_madikeri, scaler_madikeri = process_data(file_path, WINDOW_SIZE)

if X_madikeri is not None:
    # Step 2: Build the model
    model_madikeri = build_lstm_model(input_shape=(X_madikeri.shape[1], X_madikeri.shape[2]))
    
    # EarlyStopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Step 3: Train the model
    print("Starting Madikeri model training...")
    history_madikeri = model_madikeri.fit(
        X_madikeri, 
        y_madikeri,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Step 4: Save the model and scaler
    model_save_path = os.path.join(MODELS_DIR, f"{model_name}_lstm.h5")
    scaler_save_path = os.path.join(MODELS_DIR, f"{model_name}_scaler.pkl")
    
    model_madikeri.save(model_save_path)
    with open(scaler_save_path, 'wb') as f:
        pickle.dump(scaler_madikeri, f)
        
    print(f"Successfully saved Madikeri model to: {model_save_path}")
    print(f"Successfully saved Madikeri scaler to: {scaler_save_path}")
else:
    print(f"Could not train Madikeri model. Data loading failed.")

print("\n--- Madikeri Training Complete ---")


--- Training Model for: MADIKERI ---
Loading and processing data from data\madikeri_merged.csv...
Data shape: X=(3898, 60, 4), y=(3898,)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Starting Madikeri model training...
Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - loss: 0.0282 - val_loss: 0.0276
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - loss: 0.0084 - val_loss: 0.0267
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 103ms/step - loss: 0.0090 - val_loss: 0.0263
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - loss: 0.0078 - val_loss: 0.0238
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - loss: 0.0076 - val_loss: 0.0226
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - loss: 0.0074 - val_loss: 0.0209
Epoch 7/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - loss: 0.0069 - val_loss: 0.0200
Epoch 8/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 20s 95ms/step - loss: 0.0068 - val_loss: 0.0192
Epoch 9/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - loss: 0.0064 - val_loss: 0.0183
Epoch 10/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - loss: 0.0064 - val_loss: 0.0182
Epoch 11/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - loss: 0.0061 - val_los

Successfully saved Madikeri model to: models\madikeri_lstm.h5
Successfully saved Madikeri scaler to: models\madikeri_scaler.pkl

--- Madikeri Training Complete ---


In [3]:
# --- 2. SETTINGS FOR SIRSI ---
model_name = "sirsi"
file_path = os.path.join(DATA_DIR, "sirsi_merged.csv")

print(f"\n--- Training Model for: {model_name.upper()} ---")

# Step 1: Process the data
X_sirsi, y_sirsi, scaler_sirsi = process_data(file_path, WINDOW_SIZE)

if X_sirsi is not None:
    # Step 2: Build the model
    model_sirsi = build_lstm_model(input_shape=(X_sirsi.shape[1], X_sirsi.shape[2]))
    
    # EarlyStopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Step 3: Train the model
    print("Starting Sirsi model training...")
    history_sirsi = model_sirsi.fit(
        X_sirsi, 
        y_sirsi,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Step 4: Save the model and scaler
    model_save_path = os.path.join(MODELS_DIR, f"{model_name}_lstm.h5")
    scaler_save_path = os.path.join(MODELS_DIR, f"{model_name}_scaler.pkl")
    
    model_sirsi.save(model_save_path)
    with open(scaler_save_path, 'wb') as f:
        pickle.dump(scaler_sirsi, f)
        
    print(f"Successfully saved Sirsi model to: {model_save_path}")
    print(f"Successfully saved Sirsi scaler to: {scaler_save_path}")
else:
    print(f"Could not train Sirsi model. Data loading failed.")

print("\n--- Sirsi Training Complete ---")


--- Training Model for: SIRSI ---
Loading and processing data from data\sirsi_merged.csv...
Data shape: X=(3903, 60, 4), y=(3903,)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Starting Sirsi model training...
Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 124ms/step - loss: 0.0657 - val_loss: 0.0018
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - loss: 0.0039 - val_loss: 0.0014
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - loss: 0.0031 - val_loss: 0.0011
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 9s 77ms/step - loss: 0.0027 - val_loss: 0.0012
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - loss: 0.0023 - val_loss: 0.0012
Epoch 7/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 12s 84ms/step - loss: 0.0026 - val_loss: 0.0013
Epoch 8/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - loss: 0.0022 - val_loss: 0.0011
Epoch 9/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 69ms/step - loss: 0.0020 - val_loss: 0.0011
Epoch 10/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 71ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 11/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - loss: 0.0020 - val_loss: 0.00

Successfully saved Sirsi model to: models\sirsi_lstm.h5
Successfully saved Sirsi scaler to: models\sirsi_scaler.pkl

--- Sirsi Training Complete ---


In [4]:
# --- 3. SETTINGS FOR CHIKKAMAGALURU ---
model_name = "chikkamagaluru"
file_path = os.path.join(DATA_DIR, "chikkamagaluru_merged.csv")

print(f"\n--- Training Model for: {model_name.upper()} ---")

# Step 1: Process the data
X_chikkamagaluru, y_chikkamagaluru, scaler_chikkamagaluru = process_data(file_path, WINDOW_SIZE)

if X_chikkamagaluru is not None:
    # Step 2: Build the model
    model_chikkamagaluru = build_lgitstm_model(input_shape=(X_chikkamagaluru.shape[1], X_chikkamagaluru.shape[2]))
    
    # EarlyStopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Step 3: Train the model
    print("Starting Chikkamagaluru model training...")
    history_chikkamagaluru = model_chikkamagaluru.fit(
        X_chikkamagaluru, 
        y_chikkamagaluru,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Step 4: Save the model and scaler
    model_save_path = os.path.join(MODELS_DIR, f"{model_name}_lstm.h5")
    scaler_save_path = os.path.join(MODELS_DIR, f"{model_name}_scaler.pkl")
    
    model_chikkamagaluru.save(model_save_path)
    with open(scaler_save_path, 'wb') as f:
        pickle.dump(scaler_chikkamagaluru, f)
        
    print(f"Successfully saved Chikkamagaluru model to: {model_save_path}")
    print(f"Successfully saved Chikkamagaluru scaler to: {scaler_save_path}")
else:
    print(f"Could not train Chikkamagaluru model. Data loading failed.")

print("\n--- Chikkamagaluru Training Complete ---")


--- Training Model for: CHIKKAMAGALURU ---
Loading and processing data from data\chikkamagaluru_merged.csv...
Data shape: X=(3897, 60, 4), y=(3897,)


NameError: name 'build_lgitstm_model' is not defined

In [ ]:
# --- 3. SETTINGS FOR CHIKKAMAGALURU ---
model_name = "chikkamagaluru"
file_path = os.path.join(DATA_DIR, "chikkamagaluru_merged.csv")

print(f"\n--- Training Model for: {model_name.upper()} ---")

# Step 1: Process the data
X_chikkamagaluru, y_chikkamagaluru, scaler_chikkamagaluru = process_data(file_path, WINDOW_SIZE)

if X_chikkamagaluru is not None:
    # Step 2: Build the model
    model_chikkamagaluru = build_lstm_model(input_shape=(X_chikkamagaluru.shape[1], X_chikkamagaluru.shape[2]))    
    # EarlyStopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Step 3: Train the model
    print("Starting Chikkamagaluru model training...")
    history_chikkamagaluru = model_chikkamagaluru.fit(
        X_chikkamagaluru, 
        y_chikkamagaluru,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Step 4: Save the model and scaler
    model_save_path = os.path.join(MODELS_DIR, f"{model_name}_lstm.h5")
    scaler_save_path = os.path.join(MODELS_DIR, f"{model_name}_scaler.pkl")
    
    model_chikkamagaluru.save(model_save_path)
    with open(scaler_save_path, 'wb') as f:
        pickle.dump(scaler_chikkamagaluru, f)
        
    print(f"Successfully saved Chikkamagaluru model to: {model_save_path}")
    print(f"Successfully saved Chikkamagaluru scaler to: {scaler_save_path}")
else:
    print(f"Could not train Chikkamagaluru model. Data loading failed.")

print("\n--- Chikkamagaluru Training Complete ---")


--- Training Model for: CHIKKAMAGALURU ---
Loading and processing data from data\chikkamagaluru_merged.csv...
Data shape: X=(3897, 60, 4), y=(3897,)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Starting Chikkamagaluru model training...
Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 26s 118ms/step - loss: 0.0189 - val_loss: 0.0063
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step - loss: 0.0040 - val_loss: 0.0056
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 0.0035 - val_loss: 0.0058
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - loss: 0.0034 - val_loss: 0.0052
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 18s 83ms/step - loss: 0.0030 - val_loss: 0.0053
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - loss: 0.0028 - val_loss: 0.0049
Epoch 7/50
